<a href="https://colab.research.google.com/github/joyfulspace/ADP/blob/master/07_%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_7_%EC%9C%A0%EC%A0%84%EC%9E%90%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유전자알고리즘
* 모든 규칙을 만족시켜야 할 때 사용
* 교차(crossover): 2개의 염색체를 유합하는 방법 (염색체: 솔루션을 추상화한 정보 단위)
  - 교차점 1개: 싱글 크로스오버
  - 교차점이 많다고 꼭 좋은 해를 얻는 것은 아니고 다양한 해를 얻을 수 있음
* 변이(mutation): 돌연변이 발생할 때 오히려 좋은 해가 만들어질 수 있음.
  - 지역 최적화 탈출하는 효과 있음
  - 하나의 염색체, 정보 단위에서 랜덤으로 하나의 연결점을 찾은 후, 그 부분의 정보를 의도적으로 왜곡시키는 것

* 프로세스
  1. 문제/해/해집합 정의
    - 도메인 결정도 중요
    - 도메인: 해 집합의 범위
  2. 해(솔루션)의 염색체 정의 
  3. 적합도 함수 f(x)설계 
    - 적합도 평가
    - cost function 사용: 규칙 위배에 따른 상벌 부과
    - 환경에 적응하지 못하는 염색체는 페널티가 부과되어 자연선택의 확률이 낮아져 도태되도록 되어 있음
    - 규칙 정리하고 > 패널티, 리워드 정하고 > cost function 설계
    - 이 부분에 따라 성능이 크게 달라짐
    - 우선순위 정리가 필요
    - 모든 것을 100% 만족시키는 방법은 없음
  4. 좋은 품종의 유전자 교배(교차/변이 등) 
  5. 적합한 염색체 선별 
  6. 충분한 세대교체



In [1]:
# -*- coding: utf-8 -*-

import time
import random
import math
import scipy.stats
import numpy as np

# 구간과 도메인의 정의
times = ['A', 'B', 'C', 'D', 'E', 'F']
domain = [(0, (len(times)*3)-i-1) for i in range(0, len(times)*3)]

# cost function 설계를 위한 제약조건 정리
pref = {
    1: ['B', 'C', 'A'], # 1번 사람은 B, C, A 시간대를 선호
    7: ['E', 'F'],
    11: ['A', 'B'],
    15: ['E', 'D']
}

hates = {13: [1, 7], # 13번 사람은 1번, 7번 사람과 함께 있기 싫음
         10: [11, 15],
         3: [8, 12, 10]
}

man = [range(0, 12)] # 0~11이 남자
woman = [range(12, 18)]

In [2]:
domain

[(0, 17),
 (0, 16),
 (0, 15),
 (0, 14),
 (0, 13),
 (0, 12),
 (0, 11),
 (0, 10),
 (0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [ ]:
def dormcost(vec):
  # 선호하는 시간대에 따라 점수 부여
  # 싫어하는 사람에 따라 점수 부여
  # 각 시간대에 적어도 1명은 남자
  # C시간대에 여성이 있으면 안 됨

  cost = 0

  # Create list a lot of slots
  slots = []
  for i in range(len(times)): slots